# Análise Principal dos Dados Olist

Este notebook contém as análises principais focadas em 4 perguntas específicas sobre os dados da Olist.

## Configuração Inicial e Importação das Bibliotecas

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# Configuração dos gráficos
plt.style.use('default')
sns.set_palette("husl")

print("Bibliotecas importadas com sucesso!")

## Carregamento dos Dados

In [ ]:
# Carregamento dos datasets da Olist
# Substitua os caminhos pelos caminhos corretos dos seus arquivos

try:
    orders = pd.read_csv('data/olist_orders_dataset.csv')
    order_items = pd.read_csv('data/olist_order_items_dataset.csv')
    order_payments = pd.read_csv('data/olist_order_payments_dataset.csv')
    order_reviews = pd.read_csv('data/olist_order_reviews_dataset.csv')
    products = pd.read_csv('data/olist_products_dataset.csv')
    product_translation = pd.read_csv('data/product_category_name_translation.csv')
    
    print("Todos os datasets carregados com sucesso!")
    print(f"Orders: {orders.shape}")
    print(f"Order Items: {order_items.shape}")
    print(f"Order Payments: {order_payments.shape}")
    print(f"Order Reviews: {order_reviews.shape}")
    print(f"Products: {products.shape}")
    print(f"Product Translation: {product_translation.shape}")
    
except FileNotFoundError as e:
    print(f"Erro ao carregar arquivo: {e}")
    print("Por favor, verifique se os arquivos estão no diretório 'data/'")

## Pré-processamento dos Dados

In [ ]:
# Conversão de datas
date_columns = [
    'order_purchase_timestamp',
    'order_approved_at',
    'order_delivered_carrier_date',
    'order_delivered_customer_date',
    'order_estimated_delivery_date'
]

for col in date_columns:
    if col in orders.columns:
        orders[col] = pd.to_datetime(orders[col])

# Merge do translation para ter nomes das categorias em português
products_translated = products.merge(
    product_translation, 
    on='product_category_name', 
    how='left'
)

print("Pré-processamento concluído!")

---
# ANÁLISE 1: Percentual de Pedidos Entregues Após a Data Estimada

**Pergunta:** Qual o percentual de pedidos entregues após a data estimada pela Olist?

In [ ]:
# Filtrar apenas pedidos que foram entregues
delivered_orders = orders[
    (orders['order_status'] == 'delivered') & 
    (orders['order_delivered_customer_date'].notna()) & 
    (orders['order_estimated_delivery_date'].notna())
].copy()

# Calcular se a entrega foi atrasada
delivered_orders['delivery_delay'] = (
    delivered_orders['order_delivered_customer_date'] > 
    delivered_orders['order_estimated_delivery_date']
)

# Calcular percentual
total_delivered = len(delivered_orders)
delayed_deliveries = delivered_orders['delivery_delay'].sum()
percentage_delayed = (delayed_deliveries / total_delivered) * 100

print(f"Total de pedidos entregues analisados: {total_delivered:,}")
print(f"Pedidos entregues após a data estimada: {delayed_deliveries:,}")
print(f"Percentual de pedidos entregues após a data estimada: {percentage_delayed:.2f}%")

In [ ]:
# Visualização dos resultados
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

# Gráfico de pizza
labels = ['No Prazo', 'Atrasado']
sizes = [total_delivered - delayed_deliveries, delayed_deliveries]
colors = ['#2ecc71', '#e74c3c']
explode = (0, 0.1)  # explode the delayed slice

ax1.pie(sizes, labels=labels, colors=colors, autopct='%1.1f%%', 
        startangle=90, explode=explode, shadow=True)
ax1.set_title('Distribuição de Entregas: No Prazo vs Atrasadas', fontsize=14, fontweight='bold')

# Gráfico de barras
ax2.bar(labels, sizes, color=colors, alpha=0.8)
ax2.set_title('Quantidade de Entregas por Status', fontsize=14, fontweight='bold')
ax2.set_ylabel('Número de Pedidos')
ax2.yaxis.set_major_formatter(plt.FuncFormatter(lambda x, p: f'{x:,.0f}'))

# Adicionar valores nas barras
for i, v in enumerate(sizes):
    ax2.text(i, v + max(sizes) * 0.01, f'{v:,}\n({v/total_delivered*100:.1f}%)', 
             ha='center', va='bottom', fontweight='bold')

plt.tight_layout()
plt.show()

### Resposta da Análise 1:
**{percentage_delayed:.2f}%** dos pedidos foram entregues após a data estimada pela Olist.

---
# ANÁLISE 2: Método de Pagamento Mais Utilizado em Pedidos Acima de R$ 150,00

**Pergunta:** Qual o método de pagamento mais utilizado em pedidos acima de R$ 150,00?

In [ ]:
# Calcular valor total por pedido
order_values = order_payments.groupby('order_id')['payment_value'].sum().reset_index()
order_values.columns = ['order_id', 'total_payment_value']

# Filtrar pedidos acima de R$ 150
high_value_orders = order_values[order_values['total_payment_value'] > 150]

# Merge com dados de pagamento para obter métodos
high_value_payments = high_value_orders.merge(order_payments, on='order_id')

# Contar métodos de pagamento
payment_methods = high_value_payments.groupby('payment_type')['order_id'].nunique().reset_index()
payment_methods.columns = ['payment_type', 'order_count']
payment_methods = payment_methods.sort_values('order_count', ascending=False)

# Método mais utilizado
most_used_method = payment_methods.iloc[0]

print(f"Total de pedidos acima de R$ 150,00: {len(high_value_orders):,}")
print(f"\nMétodos de pagamento em pedidos acima de R$ 150,00:")
for _, row in payment_methods.iterrows():
    percentage = (row['order_count'] / len(high_value_orders)) * 100
    print(f"  {row['payment_type']}: {row['order_count']:,} pedidos ({percentage:.1f}%)")

print(f"\n✨ Método de pagamento mais utilizado: {most_used_method['payment_type']} ({most_used_method['order_count']:,} pedidos)")

In [ ]:
# Visualização dos métodos de pagamento
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))

# Gráfico de barras
colors = sns.color_palette("husl", len(payment_methods))
bars = ax1.bar(payment_methods['payment_type'], payment_methods['order_count'], color=colors, alpha=0.8)
ax1.set_title('Métodos de Pagamento em Pedidos > R$ 150,00', fontsize=14, fontweight='bold')
ax1.set_xlabel('Método de Pagamento')
ax1.set_ylabel('Número de Pedidos')
ax1.tick_params(axis='x', rotation=45)
ax1.yaxis.set_major_formatter(plt.FuncFormatter(lambda x, p: f'{x:,.0f}'))

# Adicionar valores nas barras
for bar in bars:
    height = bar.get_height()
    ax1.text(bar.get_x() + bar.get_width()/2., height + max(payment_methods['order_count']) * 0.01,
             f'{height:,.0f}', ha='center', va='bottom', fontweight='bold')

# Gráfico de pizza
ax2.pie(payment_methods['order_count'], labels=payment_methods['payment_type'], 
        colors=colors, autopct='%1.1f%%', startangle=90, shadow=True)
ax2.set_title('Distribuição dos Métodos de Pagamento\n(Pedidos > R$ 150,00)', 
              fontsize=14, fontweight='bold')

plt.tight_layout()
plt.show()

### Resposta da Análise 2:
O método de pagamento mais utilizado em pedidos acima de R$ 150,00 é **{most_used_method['payment_type']}**, representando **{(most_used_method['order_count'] / len(high_value_orders)) * 100:.1f}%** dos pedidos nessa faixa de valor.

---
# ANÁLISE 3: Relação entre Tempo de Entrega e Nota de Avaliação

**Pergunta:** Qual é a relação entre o tempo de entrega e a nota de avaliação do cliente?

In [ ]:
# Merge entre orders e reviews
orders_reviews = orders.merge(order_reviews, on='order_id', how='inner')

# Filtrar apenas pedidos entregues com reviews
delivered_with_reviews = orders_reviews[
    (orders_reviews['order_status'] == 'delivered') & 
    (orders_reviews['order_delivered_customer_date'].notna()) & 
    (orders_reviews['order_purchase_timestamp'].notna()) & 
    (orders_reviews['review_score'].notna())
].copy()

# Calcular tempo de entrega em dias
delivered_with_reviews['delivery_time_days'] = (
    delivered_with_reviews['order_delivered_customer_date'] - 
    delivered_with_reviews['order_purchase_timestamp']
).dt.days

# Remover outliers (tempo de entrega muito alto)
q95 = delivered_with_reviews['delivery_time_days'].quantile(0.95)
delivered_with_reviews = delivered_with_reviews[delivered_with_reviews['delivery_time_days'] <= q95]

# Estatísticas por nota
delivery_stats = delivered_with_reviews.groupby('review_score')['delivery_time_days'].agg([
    'count', 'mean', 'median', 'std'
]).round(2)
delivery_stats.columns = ['Quantidade', 'Tempo Médio', 'Tempo Mediano', 'Desvio Padrão']

print("Estatísticas de tempo de entrega por nota de avaliação:")
print(delivery_stats)

# Correlação
correlation = delivered_with_reviews['delivery_time_days'].corr(delivered_with_reviews['review_score'])
print(f"\nCorrelação entre tempo de entrega e nota de avaliação: {correlation:.4f}")

In [ ]:
# Visualizações da relação tempo de entrega vs avaliação
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(16, 12))

# 1. Boxplot do tempo de entrega por nota
delivered_with_reviews.boxplot(column='delivery_time_days', by='review_score', ax=ax1)
ax1.set_title('Distribuição do Tempo de Entrega por Nota de Avaliação')
ax1.set_xlabel('Nota de Avaliação')
ax1.set_ylabel('Tempo de Entrega (dias)')

# 2. Gráfico de dispersão
ax2.scatter(delivered_with_reviews['delivery_time_days'], delivered_with_reviews['review_score'], 
            alpha=0.5, s=10)
ax2.set_xlabel('Tempo de Entrega (dias)')
ax2.set_ylabel('Nota de Avaliação')
ax2.set_title(f'Correlação: Tempo de Entrega vs Nota (r = {correlation:.3f})')

# Linha de tendência
z = np.polyfit(delivered_with_reviews['delivery_time_days'], delivered_with_reviews['review_score'], 1)
p = np.poly1d(z)
ax2.plot(delivered_with_reviews['delivery_time_days'], p(delivered_with_reviews['delivery_time_days']), 
         "r--", alpha=0.8)

# 3. Tempo médio por nota
mean_delivery_time = delivered_with_reviews.groupby('review_score')['delivery_time_days'].mean()
colors = plt.cm.RdYlGn_r(np.linspace(0.2, 0.8, len(mean_delivery_time)))
bars = ax3.bar(mean_delivery_time.index, mean_delivery_time.values, color=colors, alpha=0.8)
ax3.set_xlabel('Nota de Avaliação')
ax3.set_ylabel('Tempo Médio de Entrega (dias)')
ax3.set_title('Tempo Médio de Entrega por Nota de Avaliação')

# Adicionar valores nas barras
for bar in bars:
    height = bar.get_height()
    ax3.text(bar.get_x() + bar.get_width()/2., height + 0.5,
             f'{height:.1f}', ha='center', va='bottom', fontweight='bold')

# 4. Distribuição de notas por faixa de tempo
delivered_with_reviews['delivery_range'] = pd.cut(
    delivered_with_reviews['delivery_time_days'], 
    bins=[0, 7, 14, 21, 30, 100], 
    labels=['≤ 7 dias', '8-14 dias', '15-21 dias', '22-30 dias', '> 30 dias']
)

delivery_range_scores = pd.crosstab(delivered_with_reviews['delivery_range'], 
                                   delivered_with_reviews['review_score'], 
                                   normalize='index') * 100

delivery_range_scores.plot(kind='bar', stacked=True, ax=ax4, 
                          colormap='RdYlGn', alpha=0.8)
ax4.set_title('Distribuição de Notas por Faixa de Tempo de Entrega (%)')
ax4.set_xlabel('Faixa de Tempo de Entrega')
ax4.set_ylabel('Percentual (%)')
ax4.legend(title='Nota', bbox_to_anchor=(1.05, 1), loc='upper left')
ax4.tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

### Resposta da Análise 3:
A correlação entre tempo de entrega e nota de avaliação é **{correlation:.4f}**, indicando uma relação **{'negativa moderada' if correlation < -0.3 else 'negativa fraca' if correlation < -0.1 else 'muito fraca'}**.

**Principais insights:**
- Pedidos com tempo de entrega mais curto tendem a receber notas mais altas
- O tempo médio de entrega para notas 5 é menor que para notas 1
- Entregas rápidas (≤ 7 dias) apresentam maior percentual de notas altas

---
# ANÁLISE 4: Top 5 Categorias Mais Vendidas e Receita Total

**Pergunta:** Quais são as 5 categorias de produtos mais vendidas e qual a receita total gerada por cada uma?

In [ ]:
# Merge dos dados necessários
items_products = order_items.merge(products_translated, on='product_id', how='left')

# Usar nome em português quando disponível, senão usar original
items_products['category_name'] = items_products['product_category_name_english'].fillna(
    items_products['product_category_name']
)

# Remover categorias nulas
items_products = items_products[items_products['category_name'].notna()]

# Calcular métricas por categoria
category_stats = items_products.groupby('category_name').agg({
    'order_item_id': 'count',  # Quantidade vendida
    'price': 'sum',            # Receita total
    'order_id': 'nunique'      # Número de pedidos únicos
}).round(2)

category_stats.columns = ['Quantidade_Vendida', 'Receita_Total', 'Pedidos_Unicos']
category_stats = category_stats.sort_values('Quantidade_Vendida', ascending=False)

# Top 5 categorias mais vendidas
top5_categories = category_stats.head(5)

print("🏆 TOP 5 CATEGORIAS MAIS VENDIDAS:")
print("=" * 60)
for i, (category, row) in enumerate(top5_categories.iterrows(), 1):
    print(f"{i}. {category}:")
    print(f"   📦 Quantidade vendida: {row['Quantidade_Vendida']:,} itens")
    print(f"   💰 Receita total: R$ {row['Receita_Total']:,.2f}")
    print(f"   🛒 Pedidos únicos: {row['Pedidos_Unicos']:,}")
    print()

# Receita total das top 5
total_revenue_top5 = top5_categories['Receita_Total'].sum()
total_revenue_all = category_stats['Receita_Total'].sum()
percentage_top5 = (total_revenue_top5 / total_revenue_all) * 100

print(f"💎 Receita total das TOP 5: R$ {total_revenue_top5:,.2f}")
print(f"📊 Representa {percentage_top5:.1f}% da receita total")

In [ ]:
# Visualizações das top 5 categorias
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(18, 14))

# 1. Gráfico de barras - Quantidade vendida
colors = sns.color_palette("viridis", 5)
bars1 = ax1.barh(range(5), top5_categories['Quantidade_Vendida'].values[::-1], color=colors)
ax1.set_yticks(range(5))
ax1.set_yticklabels(top5_categories.index[::-1])
ax1.set_xlabel('Quantidade Vendida')
ax1.set_title('TOP 5 Categorias - Quantidade de Itens Vendidos', fontsize=14, fontweight='bold')
ax1.xaxis.set_major_formatter(plt.FuncFormatter(lambda x, p: f'{x:,.0f}'))

# Adicionar valores nas barras
for i, bar in enumerate(bars1):
    width = bar.get_width()
    ax1.text(width + max(top5_categories['Quantidade_Vendida']) * 0.01, bar.get_y() + bar.get_height()/2,
             f'{width:,.0f}', ha='left', va='center', fontweight='bold')

# 2. Gráfico de barras - Receita total
bars2 = ax2.barh(range(5), top5_categories['Receita_Total'].values[::-1], color=colors)
ax2.set_yticks(range(5))
ax2.set_yticklabels(top5_categories.index[::-1])
ax2.set_xlabel('Receita Total (R$)')
ax2.set_title('TOP 5 Categorias - Receita Total', fontsize=14, fontweight='bold')
ax2.xaxis.set_major_formatter(plt.FuncFormatter(lambda x, p: f'R$ {x:,.0f}'))

# Adicionar valores nas barras
for i, bar in enumerate(bars2):
    width = bar.get_width()
    ax2.text(width + max(top5_categories['Receita_Total']) * 0.01, bar.get_y() + bar.get_height()/2,
             f'R$ {width:,.0f}', ha='left', va='center', fontweight='bold')

# 3. Gráfico de pizza - Participação na receita
other_revenue = total_revenue_all - total_revenue_top5
revenue_data = list(top5_categories['Receita_Total'].values) + [other_revenue]
labels_revenue = list(top5_categories.index) + ['Outras Categorias']
colors_pie = colors + ['lightgray']

wedges, texts, autotexts = ax3.pie(revenue_data, labels=labels_revenue, colors=colors_pie, 
                                  autopct='%1.1f%%', startangle=90, pctdistance=0.85)
ax3.set_title('Participação na Receita Total', fontsize=14, fontweight='bold')

# Melhorar legibilidade
for autotext in autotexts:
    autotext.set_color('white')
    autotext.set_fontweight('bold')

# 4. Receita média por item
top5_categories['Receita_Media_Item'] = top5_categories['Receita_Total'] / top5_categories['Quantidade_Vendida']
bars4 = ax4.bar(range(5), top5_categories['Receita_Media_Item'].values, color=colors, alpha=0.8)
ax4.set_xticks(range(5))
ax4.set_xticklabels([label[:15] + '...' if len(label) > 15 else label for label in top5_categories.index], 
                   rotation=45, ha='right')
ax4.set_ylabel('Receita Média por Item (R$)')
ax4.set_title('Receita Média por Item - TOP 5 Categorias', fontsize=14, fontweight='bold')
ax4.yaxis.set_major_formatter(plt.FuncFormatter(lambda x, p: f'R$ {x:.0f}'))

# Adicionar valores nas barras
for i, bar in enumerate(bars4):
    height = bar.get_height()
    ax4.text(bar.get_x() + bar.get_width()/2., height + max(top5_categories['Receita_Media_Item']) * 0.01,
             f'R$ {height:.0f}', ha='center', va='bottom', fontweight='bold')

plt.tight_layout()
plt.show()

### Resposta da Análise 4:

**🏆 As 5 categorias de produtos mais vendidas são:**

1. **{top5_categories.index[0]}** - {top5_categories.iloc[0]['Quantidade_Vendida']:,} itens vendidos, receita de R$ {top5_categories.iloc[0]['Receita_Total']:,.2f}
2. **{top5_categories.index[1]}** - {top5_categories.iloc[1]['Quantidade_Vendida']:,} itens vendidos, receita de R$ {top5_categories.iloc[1]['Receita_Total']:,.2f}
3. **{top5_categories.index[2]}** - {top5_categories.iloc[2]['Quantidade_Vendida']:,} itens vendidos, receita de R$ {top5_categories.iloc[2]['Receita_Total']:,.2f}
4. **{top5_categories.index[3]}** - {top5_categories.iloc[3]['Quantidade_Vendida']:,} itens vendidos, receita de R$ {top5_categories.iloc[3]['Receita_Total']:,.2f}
5. **{top5_categories.index[4]}** - {top5_categories.iloc[4]['Quantidade_Vendida']:,} itens vendidos, receita de R$ {top5_categories.iloc[4]['Receita_Total']:,.2f}

**💰 Receita total das TOP 5:** R$ {total_revenue_top5:,.2f} ({percentage_top5:.1f}% da receita total)

---
## 📋 Resumo Executivo das Análises

### 🎯 Principais Descobertas:

1. **Pontualidade das Entregas**: {percentage_delayed:.1f}% dos pedidos foram entregues após a data estimada

2. **Pagamentos Premium**: O método mais usado em pedidos >R$150 é **{most_used_method['payment_type']}**

3. **Satisfação vs Entrega**: Correlação de {correlation:.3f} entre tempo de entrega e nota de avaliação

4. **Top Categorias**: As 5 categorias mais vendidas geram R$ {total_revenue_top5/1000000:.1f}M em receita

### 🚀 Recomendações Estratégicas:
- Melhorar logística para reduzir entregas em atraso
- Focar nos métodos de pagamento preferidos dos clientes premium
- Investir em entregas mais rápidas para melhorar satisfação
- Concentrar esforços de marketing nas categorias top performers
